In [1]:
import os

BASE_DIR = os.getcwd() # Ou spécifiez un chemin absolu
DATA_FOLDER = os.path.join(BASE_DIR, "data", "BULLETINS")
OUTPUT_FOLDER = os.path.join(BASE_DIR, "output")

# TD2
XML_INITIAL_FILE = os.path.join(OUTPUT_FOLDER, "corpus_initial.xml")

# TD3
XML_FILTRE_FILE = os.path.join(OUTPUT_FOLDER, "corpus_filtre.xml")
SEGMENTATION_FILE = os.path.join(OUTPUT_FOLDER, "segmentation.tsv")
TF_FILE = os.path.join(OUTPUT_FOLDER, "tf.tsv")
IDF_FILE = os.path.join(OUTPUT_FOLDER, "idf.tsv")
TFIDF_FILE = os.path.join(OUTPUT_FOLDER, "tfidf.tsv")
ANTI_DICT_FILE = os.path.join(OUTPUT_FOLDER, "anti_dictionnaire.txt")

# TD4
FILTERED_XML_FILE = os.path.join(OUTPUT_FOLDER, "corpus_filtre.xml")
INDEX_OUTPUT_DIR = os.path.join(OUTPUT_FOLDER, "index_files")
COMPARISON_OUTPUT_DIR = os.path.join(OUTPUT_FOLDER, "lemma_comparison")

STEMMED_REPLACEMENT = os.path.join(OUTPUT_FOLDER, "stemmed_replacement.tsv")
INDEX_FILE = os.path.join(INDEX_OUTPUT_DIR, "index.xml")

## Load from raw documents

In [2]:
from index import BS4Parser, Corpus

file_client = BS4Parser()
documents = file_client.process_folder(DATA_FOLDER, limit=10)  # Process all files inside the folder
Corpus(documents = documents)

Corpus(documents=[Document(fichier='67068.htm', numero='258', date=datetime.datetime(2011, 6, 21, 0, 0), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physi

## Save and reload from xml

In [3]:
STORAGE_TAGS = {"Corpus": "corpus", "documents": "bulletins", "Document": "bulletin", "Image": "image"}
with open(XML_INITIAL_FILE, "w", encoding="utf-8") as file:
    file.write(Corpus(documents=documents).model_dump_xml_str_pretty(tags=STORAGE_TAGS))
with open(XML_INITIAL_FILE, "r", encoding="utf-8") as file:
    CORPUS = Corpus.model_validate_xml(file.read(), tags=STORAGE_TAGS)
CORPUS

Corpus(documents=[Document(fichier='67068.htm', numero='258', date=datetime.datetime(2011, 6, 21, 0, 0), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physi

## Tokenize documents

In [4]:
import pandas as pd
CORPUS.token_by_doc.to_csv(SEGMENTATION_FILE, sep='\t', index=False, header=False, encoding='utf-8')
pd.read_csv(SEGMENTATION_FILE, sep='\t', header=None, names=["Document", "Segment"]).head()

,Document,Segment
0,physique,67068.htm
1,mathias,67068.htm
2,fink,67068.htm
3,un,67068.htm
4,bel,67068.htm


## Compute tf/logtf metrics

In [5]:
CORPUS.term_frequencies.head()

,document_id,mot,tf
0,67068.htm,physique,10
1,67068.htm,mathias,10
2,67068.htm,fink,10
3,67068.htm,un,13
4,67068.htm,bel,1


In [6]:
with open(ANTI_DICT_FILE, "w+", encoding="utf-8") as file:
    file.writelines([f"{token}\t\"\"\n" for token in CORPUS.anti_dict()])
print(f"{CORPUS.anti_dict().iloc[0]}\t\"\"\n")

a	""



## Use Anti Dict

In [7]:
FILTERED_CORPUS = CORPUS.model_copy(deep=True)
FILTERED_CORPUS.apply_substitutions(ANTI_DICT_FILE)

Application des substitutions depuis g:\GitHub\UTC\LO17\PremierDM\output\anti_dictionnaire.txt sur 10 documents...
  ... 10/10 documents traités.
Substitutions appliquées. 10 documents ont été modifiés.
Le cache des documents a été correctement invalidé pour les documents modifiés.


In [8]:
print(CORPUS.documents[0].corps)
print("---")
print(FILTERED_CORPUS.documents[0].corps)

Physique : Mathias Fink, un bel exemple de chercheur qui innove
Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l'Innovation, dont Valérie Pécresse, ministre de l'Enseignement Supérieur et de la Recherche est à l'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l'économiste Esther Duflo, le roboticien François Pierrot et le physicien Mathias Fink. Directeur de l'Institut Langevin "Ondes et Images", créé en janvier 2009 au sein de l'Ecole de Physique et de Chimie Industrielles de la Ville de Paris (ESPCI), Mathias Fink est un remarquable exemple de ces chercheurs qui innovent. Les travaux de son équipe ont en effet abouti à la création de quatre start-ups, qui tot

## Stemming
(TD4)

In [9]:
snowball_stems = FILTERED_CORPUS.snowball_stems
spacy_lemmas = FILTERED_CORPUS.spacy_lemmas

pd.DataFrame({
    "Token": list(snowball_stems["word"]), 
    "Spacy": list(spacy_lemmas["stem"]), 
    "Snowball": list(snowball_stems["stem"]),
}).head()

,Token,Spacy,Snowball
0,abattus,abattu,abattus
1,abord,abord,abord
2,abouti,aboutir,about
3,aboutir,aboutir,about
4,absorber,absorber,absorb


In [10]:
pd.DataFrame({
    "total_mots": [len(snowball_stems["stem"])] * 2,
    "racines_uniques": [x["stem"].nunique() for x in (spacy_lemmas, snowball_stems)],
    "mots_non_changes": [x[x["word"] == x["stem"]].shape[0] for x in (spacy_lemmas, snowball_stems)]
}, index=["Spacy", "Snowball"]).T

,Spacy,Snowball
total_mots,1554,1554
racines_uniques,1283,1225
mots_non_changes,869,397


## Correcteur

In [11]:
from index import Correcteur

lexique = list(CORPUS.tokens.keys())
CORRECTEUR = Correcteur(lexique)

CORRECTEUR.process_sentence("bonjour j'aime les dromadaires et les chameaux de Pécras", max_overflow=4)

['bon', None, 'les', None, 'et', 'les', 'champ', 'de', 'pécresse']

In [12]:
CORRECTEUR.search_tree(Correcteur(lexique).prefix_tree, "piaep")

(['pierrot', 'pierre', 'pièce', 'pilote', 'piste', 'pistes'], 2)

## Recherche dans un Index

In [13]:
stems = spacy_lemmas
with open(STEMMED_REPLACEMENT, "w+", encoding="utf-8") as file:
    file.writelines([f"{word}\t\"{stem}\n" for word, stem in zip(stems['word'], stems['stem'])])
    
FILTERED_CORPUS.apply_substitutions(STEMMED_REPLACEMENT)

Application des substitutions depuis g:\GitHub\UTC\LO17\PremierDM\output\stemmed_replacement.tsv sur 10 documents...
  ... 10/10 documents traités.
Substitutions appliquées. 10 documents ont été modifiés.
Le cache des documents a été correctement invalidé pour les documents modifiés.


In [14]:
# Construction de l'index
from typing import List, Dict

INDEX: Dict[str, Dict[str, List[str]]] = {
    "texte": {},
    "titre": {},
    "rubrique": {},
    "date": {},
}

for token in FILTERED_CORPUS.tokens.keys():
    for doc in FILTERED_CORPUS.documents:
        if token in doc.texte:
            INDEX["texte"].setdefault(token, []).append(doc.document_id)
        if token in doc.titre:
            INDEX["titre"].setdefault(token, []).append(doc.document_id)
        if token in doc.rubrique:
            INDEX["rubrique"].setdefault(token, []).append(doc.document_id)

for doc in FILTERED_CORPUS.documents:        
    INDEX["date"].setdefault(doc.date, []).append(doc.document_id)
        
list(INDEX["texte"].keys())[:5]

['physique', 'mathia', 'fink', 'bel', 'exemple']

In [37]:
from typing import List
from index import Document

CORRECTEUR = Correcteur(lexique)
ANTI_DICT = CORPUS.anti_dict()

def tokenize(text: str) -> List[str]:
    tokens = CORRECTEUR.process_sentence(text, max_overflow=4)
    relevant_tokens = [token for token in tokens if token]
    lemmatized_tokens = FILTERED_CORPUS.spacy_lemmatize(" ".join(relevant_tokens))
    return [token for token in lemmatized_tokens if token not in ANTI_DICT]

def get_doc(doc_id: str) -> Document:
    """
    Récupère un document à partir de son ID.
    """
    for doc in CORPUS.documents:
        if doc.document_id == doc_id:
            return doc
    return None

def search(input: str) -> List[Document]:
    tokens = tokenize(input)
    print(f"Tokens: {tokens}")
    
    relevant_docs = {key: set() for key in INDEX.keys()}
    
    for field, token_mapping in INDEX.items():
        for token, doc_ids in token_mapping.items():
            if token in tokens:
                relevant_docs[field].update(doc_ids)

    return relevant_docs

search("Bonjour j'aime les carottes")

Tokens: ['bon', 'le', 'carbone']


{'texte': {'67385.htm', '67386.htm', '67387.htm'},
 'titre': set(),
 'rubrique': set(),
 'date': set()}